<h1>코드 한 셀 요약</h1>

아래 내용 실행 후 셀 하나만 돌리면 됩니다. Settings 이후로는 기록용!
1. 토큰 path 확인
2. csv 파일 저장위치 확인

In [7]:
pwd

'c:\\Users\\yenny\\github\\Brawlstars_project'

In [1]:
import brawlstats
from pathlib import Path
import pandas as pd
import datetime

token = open(Path('C:/Users/yenny/Desktop/brawlstats_token_skrrfee.txt')).read().strip()

now = datetime.datetime.now()
time = now.strftime('%Y%m%d_%H%M')

client = brawlstats.Client(token)
rankers = client.get_rankings(ranking='players')
rankers_tag_list = [rankers[i]['tag'].split("#")[1] for i in range(len(rankers))]
users_tag_list = ['GPYC8RPGU', 'G0CUVGVLP', 'GOCUCCG8P', 'Q89CUGU0P', 'YR2J2QUL', '9J20P02VR', '92C0JPU2Q']

## 23-03-06 듀오쇼다운 모드 수집하는 코드 추가

def parse_single_user(tag):
    parsed_logs = []
    battles = client.get_battle_logs(tag)
    for battle in battles:

        # 배틀모드가 3대3인 경우만 추출합니다. 그외 클럽리그, 파워리그, 보스전, 빅게임, 로보럼블까지 총 5개의 모드는 배틀로그 형태가 달라서 일단 제외합니다.
        if battle['event']['mode'] in ['gemGrab', 'brawlBall', 'knockout', 'heist', 'bounty', 'hotZone', 'basketBrawl']:

            battle_info = {
                "battle_time": battle['battle_time'],
                "mode": battle['event']['mode'],
                "map": battle['event']['map'],
                "type": battle['battle']['type'],
                "duration": battle['battle']['duration'],
            }

            # 배틀결과가 무승부인 경우와 같이 스타플레이어가 없는 경우 오류를 피하기 위해 if 조건문을 사용합니다.
            if battle['battle']['star_player'] != None:
                try:
                    star_player_tag = battle['battle']['star_player']['tag'][1:]
                except:
                    pass

                teams = battle['battle']['teams']

                for team in teams:
                    players_tag = [member['tag'][1:] for member in team]
                    side = 1 if tag in players_tag else 2

                    for player in team:
                        player_info = client.get_profile(player['tag'][1:])
                        player_brawlers = sorted(player_info.brawlers, key= lambda x: x['trophies'], reverse=True)

                        for brawler in player_brawlers:
                            if brawler['id']==player['brawler']['id']:
                                gadget = 1 if ['gadgets'] else 0
                                star_power = 1 if brawler['star_powers'] else 0
                        
                        team_info = {
                            'player_tag': player['tag'],
                            'player_name': player['name'],
                            'player_trophies': player_info.trophies,
                            'player_top_brawler': player_brawlers[0]['name'],
                            'star_player': 1 if player['tag'][1:] == star_player_tag else 0,
                            'team': side,
                            'result': battle['battle']['result'],
                            'rank': None,
                            'brawler_id': player['brawler']['id'],
                            'brawler_name': player['brawler']['name'],
                            'brawler_power': player['brawler']['power'],
                            'brawler_star_power': star_power,
                            'brawler_gadget': gadget,
                            'brawler_trophies': player['brawler']['trophies'],
                        }

                        parsed_logs.append({**battle_info, **team_info})

        elif battle['event']['mode'] == 'duoShowdown':
            
            battle_info = {
                "battle_time": battle['battle_time'],
                "mode": battle['event']['mode'],
                "map": battle['event']['map'],
                "type": battle['battle']['type'],
                "duration": None,
            }

            teams = battle['battle']['teams']

            for team in teams:
                players_tag = [member['tag'][1:] for member in team]
                side = 1 if tag in players_tag else teams.index(team) + 2

                for player in team:
                    player_info = client.get_profile(player['tag'][1:])
                    player_brawlers = sorted(player_info.brawlers, key= lambda x: x['trophies'], reverse=True)

                    for brawler in player_brawlers:
                        if brawler['id']==player['brawler']['id']:
                            gadget = 1 if ['gadgets'] else 0
                            star_power = 1 if brawler['star_powers'] else 0
                    
                    team_info = {
                        'player_tag': player['tag'],
                        'player_name': player['name'],
                        'player_trophies': player_info.trophies,
                        'player_top_brawler': player_brawlers[0]['name'],
                        'star_player': 1 if player['tag'][1:] == star_player_tag else 0,
                        'team': side,
                        'result': None,
                        'rank': battle['battle']['rank'] if tag in players_tag else 0,
                        'brawler_id': player['brawler']['id'],
                        'brawler_name': player['brawler']['name'],
                        'brawler_power': player['brawler']['power'],
                        'brawler_star_power': star_power,
                        'brawler_gadget': gadget,
                        'brawler_trophies': player['brawler']['trophies'],
                    }

                    parsed_logs.append({**battle_info, **team_info})
    

    return parsed_logs

columns = pd.DataFrame(parse_single_user(rankers_tag_list[0])).columns
df = pd.DataFrame(columns=columns)

for ranker in rankers_tag_list:
    try:
        df_temp = pd.DataFrame(parse_single_user(ranker))
        df = pd.concat([df, df_temp], axis=0, ignore_index=True)
    except:
        pass

df_origin = df.copy()

# player_name 값에서 숫자, 영문만 남깁니다.
df["player_name"] = df["player_name"].str.replace(pat=r'[^1-9a-zA-Z]', repl=r'', regex=True)

if df.shape[1] == 19:
    df.to_csv(f'C:/Users/yenny/Desktop/{time}_rankers.csv', encoding='utf-8', index=False)
else:
    print('랭커의 배틀로그를 저장하지 못했습니다. 데이터프레임 열을 확인해 주세요.')


## Users 배틀로그 저장
df = pd.DataFrame(columns=columns)
for user in users_tag_list:
    try:
        df_temp = pd.DataFrame(parse_single_user(user))
        df = pd.concat([df, df_temp], axis=0, ignore_index=True)
    except:
        pass

df_origin = df.copy()

# player_name 값에서 숫자, 영문만 남깁니다.
df["player_name"] = df["player_name"].str.replace(pat=r'[^1-9a-zA-Z]', repl=r'', regex=True)

if df.shape[1] == 19:
    df.to_csv(f'C:/Users/yenny/Desktop/{time}_users.csv', encoding='utf-8', index=False)
else:
    print('랭커의 배틀로그를 저장하지 못했습니다. 데이터프레임 열을 확인해 주세요.')

<h1>Settings</h1>

In [ ]:
# pip install brawlstats

In [1]:
import brawlstats
from pathlib import Path
import pandas as pd
import datetime
from get_battlelogs import parse_single_user

now = datetime.datetime.now()
time = now.strftime('%Y%m%d_%H%M')

In [2]:
token = open(Path('G:/내 드라이브/GitHub/Brawl Stars Stats/brawlstats_token.txt')).read().strip()

# 클래스를 객체화합니다.
client = brawlstats.Client(token)

# 랭커의 데이터를 수집합니다. brawlstats.Client 클래스에서는 1~200위 랭커의 정보를 제공하고 있습니다.
rankers = client.get_rankings(ranking='players')

# 랭커들의 태그를 추출해서 리스트로 변환합니다.
rankers_tag_list = [rankers[i]['tag'].split("#")[1] for i in range(len(rankers))]
rankers_tag_list

['YGU20CRG',
 'VLQPVPY',
 '8820PPUC',
 'VGPLRCVQ',
 'QCC0UVC',
 '8LQ9JR82',
 'G2JQC0U',
 '8URCLRY8',
 '82UCLY9C8',
 '2QU0QU2VQ',
 '2PGGR8Y9P',
 'JGUYRYQ',
 'LLQ8GY8',
 'CJV2PJ0R',
 '8YL8889QP',
 'UYUUQQ8Q',
 'QRU0L0U',
 'CPL9PLC8',
 'CG0YQP2C',
 '9P2CUYJ8P',
 '20VYV2CQJ',
 '8GUPLYY',
 '2GJVUCL',
 'UVVULJUQ',
 '99YL0PV9V',
 'P99QGPLU2',
 'QLQCLGYV2',
 'JGCCGY80',
 '2YCR9RLCJ',
 'V2Q080UU',
 '92C8J829Q',
 '2R2CVPJYJ',
 '9C9YJ22YL',
 'CL2RUY8Y',
 '2RG0LV02',
 'Y989PY8Q',
 '90220GRPU',
 '802VG829',
 '80YGP0YQ',
 'GJ9PLGPL',
 '8JVV0P808',
 '2L892GP',
 'JLJ2LQ0Y',
 'VCJGV08V',
 '2JJ9G0UP2',
 'QRUQQLV0',
 'QQPJ2QJ9',
 '9Q9C9JU0',
 '28PP0C82J',
 '2UYJV98J2',
 'CGQCYJGY',
 '89JUJUVYL',
 '8U2GYYCR9',
 '8QPQUJUPU',
 'VYR0RR0P',
 '9P98UCUV',
 'VY20VCCY',
 'PP9R0PQ8J',
 'PR9U2JL',
 'QUYCVC2',
 '29JJCQULC',
 '808YCJQ0',
 '280RJ92JC',
 '89JJV992Y',
 '2LL9008LV',
 '2VPLLJJ0V',
 '20RVGULU2',
 'YPRJQCPJU',
 '2P088J29C',
 '2P2URUV8G',
 '8R99PJ890',
 'Y9CCULV',
 'QURVLPG',
 '200VLU82U',
 '29Q9GGJG',
 'Y98

In [3]:
# Client 클래스의 get_profile 함수를 이용하면 해당 유저의 브롤러와 가젯유무, 스타파워유무 등을 알 수 있습니다.
# 1위 랭커의 profile을 살펴봅니다.
tag = rankers_tag_list[0]
user_info = client.get_profile(tag)
user_info.brawlers

BoxList([{'id': 16000000, 'name': 'SHELLY', 'power': 11, 'rank': 35, 'trophies': 1080, 'highest_trophies': 1302, 'gears': [{'id': 62000002, 'name': 'DAMAGE', 'level': 3}, {'id': 62000000, 'name': 'SPEED', 'level': 3}, {'id': 62000004, 'name': 'SHIELD', 'level': 3}], 'star_powers': [{'id': 23000076, 'name': 'SHELL SHOCK'}, {'id': 23000135, 'name': 'BAND-AID'}], 'gadgets': [{'id': 23000255, 'name': 'FAST FORWARD'}, {'id': 23000288, 'name': 'CLAY PIGEONS'}]}, {'id': 16000001, 'name': 'COLT', 'power': 11, 'rank': 35, 'trophies': 1080, 'highest_trophies': 1344, 'gears': [{'id': 62000002, 'name': 'DAMAGE', 'level': 3}, {'id': 62000004, 'name': 'SHIELD', 'level': 3}, {'id': 62000005, 'name': 'RELOAD SPEED', 'level': 3}], 'star_powers': [{'id': 23000077, 'name': 'SLICK BOOTS'}, {'id': 23000138, 'name': 'MAGNUM SPECIAL'}], 'gadgets': [{'id': 23000273, 'name': 'SPEEDLOADER'}, {'id': 23000319, 'name': 'SILVER BULLET'}]}, {'id': 16000002, 'name': 'BULL', 'power': 11, 'rank': 35, 'trophies': 1080, 

In [4]:
# get_battle_logs 결괏값의 구조를 확인합니다.
battles = client.get_battle_logs(tag)
battle = battles[0]

print(battle)
print()
print(battle.keys())

{'battle_time': '20230306T081845.000Z', 'event': {'id': 15000024, 'mode': 'brawlBall', 'map': 'Backyard Bowl'}, 'battle': {'mode': 'brawlBall', 'type': 'ranked', 'result': 'victory', 'duration': 81, 'trophy_change': 8, 'star_player': {'tag': '#YGU20CRG', 'name': 'vital shark', 'brawler': {'id': 16000065, 'name': 'MANDY', 'power': 11, 'trophies': 684}}, 'teams': [[{'tag': '#2VYPCU0GL', 'name': 'Введи своё имя', 'brawler': {'id': 16000017, 'name': 'TARA', 'power': 7, 'trophies': 524}}, {'tag': '#PGJPLC202', 'name': 'Toskano ニャー', 'brawler': {'id': 16000005, 'name': 'SPIKE', 'power': 10, 'trophies': 629}}, {'tag': '#YGU20CRG', 'name': 'vital shark', 'brawler': {'id': 16000065, 'name': 'MANDY', 'power': 11, 'trophies': 684}}], [{'tag': '#CC8YL980', 'name': 'marco_decre💨', 'brawler': {'id': 16000011, 'name': 'MORTIS', 'power': 11, 'trophies': 678}}, {'tag': '#98YCPR9C', 'name': 'MasterX', 'brawler': {'id': 16000030, 'name': 'EMZ', 'power': 11, 'trophies': 701}}, {'tag': '#9RVJCYJJP', 'name'

<h1>함수 만들기</h1>
기록용입니다. 건너 뛰어도 무방합니다.

23-03-02 오류 : `TypeError: 'NoneType' object is not subscriptable`
- 오류 내용을 구글링
- 원인 : bounty 모드에서는 스타플레이어가 없다. 그래서 battle['battle']['star_player']가 None 값이었다.
- 해결 : 모드가 bounty 이면 star_player 정보를 넣지 않는다.

23-03-02 오류 : `TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"`
- 오류 내용을 구글링
- 원인 : 데이터프레임이 여러개인데 리스트로 넣지 않아서.. 간단하군!
- 해결 : 데이터프레임을 리스트로 묶어준다. `pd.concat([df1, df2], axis=0, sort='battle_time')` 그럼 해결!

23-03-02 오류 : 데이터수집 후 csv 파일로 저장 시 player_trophies 열의 값이 사라지고 하나씩 열이 당겨지는 현상이 발생한다.
- 원인 : player_name 문자열 가장 끝의 값이 이모티콘인 경우 발생하는 듯하다.
- 해결시도 :
1. player_name에서 이모티콘을 삭제한다. (플레이어의 태그로 식별 가능하기에 정보 자체에 문제가 발생하지는 않는다.)
2. 100% 문제해결을 위해 숫자, 영문, 한글이 아닌 경우 해당 문자를 삭제한다.

23-03-06 회의 후 결정사항
- 클럽리그, 파워리그, 보스전, 빅게임, 로보럼블, 듀오쇼다운, 솔로쇼다운 없음. -> 필요 없음.
- 23-03-06 듀오쇼다운 정보 가져와서 데이터프레임화.

23-03-06 parse_single_user 함수 수정
1. star_player 여부를 따지는 조건문에서 else 문을 삭제.
2. star_player 변수 생성을 try except 문으로 설정 `try:
                    star_player_tag = battle['battle']['star_player']['tag'][1:]
                except:
                    pass`
- 결과 : 속도 향상

<h3>브롤볼, 듀오쇼다운 코드분석</h3>

<h5>브롤볼 배틀로그</h5>

```Box(
    {'battle_time': '20230306T081033.000Z',
     'event': {'id': 15000024, 'mode': 'brawlBall', 'map': 'Backyard Bowl'}, 
     'battle': {'mode': 'brawlBall', 
                'type': 'ranked', 
                'result': 'victory', 
                'duration': 75, 
                'trophy_change': 8, 
                'star_player': {'tag': '#YGU20CRG', 'name': 'vital shark', 'brawler': {'id': 16000065, 'name': 'MANDY', 'power': 11, 'trophies': 660}}, 
                'teams': [
                            [{'tag': '#PGJPLC202', 'name': 'Toskano ニャー', 'brawler': {'id': 16000005, 'name': 'SPIKE', 'power': 10, 'trophies': 605}}, {'tag': '#9GPULLQGJ', 'name': 'Sallen❦︎', 'brawler': {'id': 16000031, 'name': 'MR. P', 'power': 11, 'trophies': 699}}, {'tag': '#YGU20CRG', 'name': 'vital shark', 'brawler': {'id': 16000065, 'name': 'MANDY', 'power': 11, 'trophies': 660}}], 
                            [{'tag': '#8LG8VJP9V', 'name': 'KuXniA', 'brawler': {'id': 16000038, 'name': 'SURGE', 'power': 10, 'trophies': 642}}, {'tag': '#80C0PYUJ', 'name': 'Szymon', 'brawler': {'id': 16000013, 'name': 'POCO', 'power': 9, 'trophies': 684}}, {'tag': '#Y29CCUCGC', 'name': 'Руслан про', 'brawler': {'id': 16000011, 'name': 'MORTIS', 'power': 11, 'trophies': 703}}]
                        ]
                    }
                }
            )```

<h5>듀오쇼다운 배틀로그</h5>

```Box(
    {'battle_time': '20230306T074656.000Z', 
     'event': {'id': 15000034, 'mode': 'duoShowdown', 'map': 'Skull Creek'}, 
     'battle': {'mode': 'duoShowdown', 
                'type': 'ranked', 
                'rank': 1, 
                'trophy_change': 5, 
                'teams': [
                            [{'tag': '#80PC2QJCR', 'name': '么Aиdŕē🖤Tøxīc メ', 'brawler': {'id': 16000009, 'name': 'DYNAMIKE', 'power': 10, 'trophies': 1002}}, {'tag': '#8R99QVPYQ', 'name': 'gunnar🩸', 'brawler': {'id': 16000063, 'name': 'CHESTER', 'power': 11, 'trophies': 997}}], 
                            [{'tag': '#QURVLPG', 'name': 'TRB | Ezlivi', 'brawler': {'id': 16000047, 'name': 'SQUEAK', 'power': 11, 'trophies': 972}}, {'tag': '#YJUGYR0P0', 'name': 'mary ✰', 'brawler': {'id': 16000029, 'name': 'BEA', 'power': 11, 'trophies': 1026}}], 
                            [{'tag': '#8GQLP2LR0', 'name': 'nancy UwU', 'brawler': {'id': 16000058, 'name': 'BONNIE', 'power': 9, 'trophies': 830}}, {'tag': '#P02829J8Q', 'name': 'ShakingCord5904', 'brawler': {'id': 16000009, 'name': 'DYNAMIKE', 'power': 10, 'trophies': 855}}], 
                            [{'tag': '#9CY899RYC', 'name': 'Ｊｏａｑｕｉｎ', 'brawler': {'id': 16000049, 'name': 'BUZZ', 'power': 10, 'trophies': 719}}, {'tag': '#8YV8CQUC8', 'name': '🔴ＲＩＣＨＡＲＤ|ＢＳ🔴', 'brawler': {'id': 16000001, 'name': 'COLT', 'power': 10, 'trophies': 735}}], 
                            [{'tag': '#G8LVUQGC', 'name': 'Lil’ Coco', 'brawler': {'id': 16000014, 'name': 'BO', 'power': 9, 'trophies': 698}}, {'tag': '#L98VL2CP', 'name': 'RagingCoconuts', 'brawler': {'id': 16000027, 'name': '8-BIT', 'power': 9, 'trophies': 698}}]
                        ]
                    }
                }
            )```

브롤볼과 비교했을 때 듀오쇼다운은 battle 밸류에 result 없음, duration 없음, rank(int) 있음, star_player 없음, teams는 총 5개 각 2명.

In [5]:
# 배틀정보를 딕셔너리로 만들고, value만 가져와서 리스트로 변환합니다.
# 다음 블로그를 참고해서 함수를 수정했습니다. (https://assaeunji.github.io/data%20analysis/2020-05-20-brawlstats/)

def parse_single_user_v1(tag):
    parsed_logs = []
    battles = client.get_battle_logs(tag)
    for battle in battles:

        # 배틀모드가 3대3인 경우만 추출합니다. 그외 클럽리그, 파워리그, 보스전, 빅게임, 로보럼블까지 총 5개의 모드는 배틀로그 형태가 달라서 일단 제외합니다.
        if battle['event']['mode'] in ['gemGrab', 'brawlBall', 'knockout', 'heist', 'bounty', 'hotZone', 'basketBrawl']:

            battle_info = {
                "battle_time": battle['battle_time'],
                "mode": battle['event']['mode'],
                "map": battle['event']['map'],
                "type": battle['battle']['type'],
                "duration": battle['battle']['duration']
            }

            # 배틀결과가 무승부인 경우와 같이 스타플레이어가 없는 경우 오류를 피하기 위해 if 조건문을 사용합니다.
            if battle['battle']['star_player'] != None:
                star_player_tag = battle['battle']['star_player']['tag'][1:]

                teams = battle['battle']['teams']

                for team in teams:
                    players_tag = [member['tag'][1:] for member in team]
                    side = 1 if tag in players_tag else 0

                    for player in team:
                        player_info = client.get_profile(player['tag'][1:])
                        player_brawlers = sorted(player_info.brawlers, key= lambda x: x['trophies'], reverse=True)

                        for brawler in player_brawlers:
                            if brawler['id']==player['brawler']['id']:
                                gadget = 1 if ['gadgets'] else 0
                                star_power = 1 if brawler['star_powers'] else 0
                        
                        team_info = {
                            'player_tag': player['tag'],
                            'player_name': player['name'],
                            'player_trophies': player_info.trophies,
                            'player_top_brawler': player_brawlers[0]['name'],
                            'star_player': 1 if player['tag'][1:] == star_player_tag else 0,
                            'team': side,
                            'result': battle['battle']['result'],
                            'brawler_id': player['brawler']['id'],
                            'brawler_name': player['brawler']['name'],
                            'brawler_power': player['brawler']['power'],
                            'brawler_star_power': star_power,
                            'brawler_gadget': gadget,
                            'brawler_trophies': player['brawler']['trophies'],
                        }

                        parsed_logs.append({**battle_info, **team_info})
            
            # 스타플레이어가 없는 경우
            else:
                teams = battle['battle']['teams']

                for team in teams:
                    players_tag = [member['tag'][1:] for member in team]
                    side = 1 if tag in players_tag else 0

                    for player in team:
                        player_info = client.get_profile(player['tag'][1:])
                        player_brawlers = sorted(player_info.brawlers, key= lambda x: x['trophies'], reverse=True)

                        for brawler in player_brawlers:
                            if brawler['id']==player['brawler']['id']:
                                gadget = 1 if ['gadgets'] else 0
                                star_power = 1 if brawler['star_powers'] else 0
                        
                        team_info = {
                            'player_tag': player['tag'],
                            'player_name': player['name'],
                            'player_trophies': player_info.trophies,
                            'player_top_brawler': player_brawlers[0]['name'],
                            'star_player':None,
                            'team': side,
                            'result': battle['battle']['result'],
                            'brawler_id': player['brawler']['id'],
                            'brawler_name': player['brawler']['name'],
                            'brawler_power': player['brawler']['power'],
                            'brawler_star_power': star_power,
                            'brawler_gadget': gadget,
                            'brawler_trophies': player['brawler']['trophies'],
                        }

                        parsed_logs.append({**battle_info, **team_info})

    return parsed_logs

In [6]:
## 230306 함수 코드 수정

# if else 문을 사용하지 않고 try except 예외 처리로 star_player가 None인 케이스를 해결했습니다.
# 함수 개발과정 기록입니다. 실제 데이터수집에 사용하는 함수는 아래에 있습니다.

def parse_single_user_v2(tag):
    parsed_logs = []
    battles = client.get_battle_logs(tag)
    for battle in battles:

        # 배틀모드가 3대3인 경우만 추출합니다. 그외 클럽리그, 파워리그, 보스전, 빅게임, 로보럼블까지 총 5개의 모드는 배틀로그 형태가 달라서 일단 제외합니다.
        if battle['event']['mode'] in ['gemGrab', 'brawlBall', 'knockout', 'heist', 'bounty', 'hotZone', 'basketBrawl']:

            battle_info = {
                "battle_time": battle['battle_time'],
                "mode": battle['event']['mode'],
                "map": battle['event']['map'],
                "type": battle['battle']['type'],
                "duration": battle['battle']['duration'],
            }

            # 배틀결과가 무승부인 경우와 같이 스타플레이어가 없는 경우 오류를 피하기 위해 if 조건문을 사용합니다.
            if battle['battle']['star_player'] != None:
                try:
                    star_player_tag = battle['battle']['star_player']['tag'][1:]
                except:
                    pass

                teams = battle['battle']['teams']

                for team in teams:
                    players_tag = [member['tag'][1:] for member in team]
                    side = 1 if tag in players_tag else 2

                    for player in team:
                        player_info = client.get_profile(player['tag'][1:])
                        player_brawlers = sorted(player_info.brawlers, key= lambda x: x['trophies'], reverse=True)

                        for brawler in player_brawlers:
                            if brawler['id']==player['brawler']['id']:
                                gadget = 1 if ['gadgets'] else 0
                                star_power = 1 if brawler['star_powers'] else 0
                        
                        team_info = {
                            'player_tag': player['tag'],
                            'player_name': player['name'],
                            'player_trophies': player_info.trophies,
                            'player_top_brawler': player_brawlers[0]['name'],
                            'star_player': 1 if player['tag'][1:] == star_player_tag else 0,
                            'team': side,
                            'result': battle['battle']['result'],
                            'brawler_id': player['brawler']['id'],
                            'brawler_name': player['brawler']['name'],
                            'brawler_power': player['brawler']['power'],
                            'brawler_star_power': star_power,
                            'brawler_gadget': gadget,
                            'brawler_trophies': player['brawler']['trophies'],
                        }

                        parsed_logs.append({**battle_info, **team_info})

    return parsed_logs

In [13]:
# 소요시간을 확인하기 위해 유저의 데이터를 불러옵니다.
tag = rankers_tag_list[0]

In [14]:
# 230306 star_player 부분 수정 전
temp = pd.DataFrame(parse_single_user_v1(tag))
print(temp['star_player'].value_counts())
print()
temp[temp['star_player']==1].player_tag

0    125
1     25
Name: star_player, dtype: int64



2       #YGU20CRG
6      #PGJPLC202
12     #PGJPLC202
20      #YGU20CRG
26      #YGU20CRG
32     #82YGQJUGG
36     #PGJPLC202
46      #YGU20CRG
52     #9GPULLQGJ
56      #YGU20CRG
63     #PGJPLC202
69     #PGJPLC202
76      #YGU20CRG
80      #YGU20CRG
89      #YGU20CRG
93     #9GPULLQGJ
100    #PGJPLC202
102    #9GPULLQGJ
112     #YGU20CRG
116     #YGU20CRG
123    #PGJPLC202
126    #PGJPLC202
137     #YGU20CRG
140     #YGU20CRG
148     #YGU20CRG
Name: player_tag, dtype: object

In [15]:
# 소요시간을 확인하기 위해 유저의 데이터를 불러옵니다. 데이터를 한 번 불러오면 재실행 시 소요시간이 단축되므로 다른 유저의 데이터를 불러옵니다.
tag = rankers_tag_list[1]

In [16]:
# 230306 star_player 부분 수정 후
temp = pd.DataFrame(parse_single_user_v2(tag))
print(temp['star_player'].value_counts())
print()
temp[temp['star_player']==1].player_tag

0    125
1     25
Name: star_player, dtype: int64



4       #GJU2P028
10      #GJU2P028
14      #GJU2P028
18     #L99QRUPQ8
28     #L99QRUPQ8
33       #VLQPVPY
39     #L99QRUPQ8
43      #GJU2P028
51     #L99QRUPQ8
54     #L99QRUPQ8
64      #GJU2P028
70      #GJU2P028
77       #VLQPVPY
80       #VLQPVPY
89       #VLQPVPY
95      #GJU2P028
98      #GJU2P028
106     #GJU2P028
109     #GJU2P028
119     #GJU2P028
125      #VLQPVPY
127     #GJU2P028
132    #L99QRUPQ8
142     #GJU2P028
146      #VLQPVPY
Name: player_tag, dtype: object

<h1>데이터 수집</h1>

In [39]:
## 23-03-06 듀오쇼다운 모드 수집하는 코드 추가

def parse_single_user(tag):
    parsed_logs = []
    battles = client.get_battle_logs(tag)
    for battle in battles:

        # 배틀모드가 3대3인 경우만 추출합니다. 그외 클럽리그, 파워리그, 보스전, 빅게임, 로보럼블까지 총 5개의 모드는 배틀로그 형태가 달라서 일단 제외합니다.
        if battle['event']['mode'] in ['gemGrab', 'brawlBall', 'knockout', 'heist', 'bounty', 'hotZone', 'basketBrawl']:

            battle_info = {
                "battle_time": battle['battle_time'],
                "mode": battle['event']['mode'],
                "map": battle['event']['map'],
                "type": battle['battle']['type'],
                "duration": battle['battle']['duration'],
            }

            # 배틀결과가 무승부인 경우와 같이 스타플레이어가 없는 경우 오류를 피하기 위해 if 조건문을 사용합니다.
            if battle['battle']['star_player'] != None:
                try:
                    star_player_tag = battle['battle']['star_player']['tag'][1:]
                except:
                    pass

                teams = battle['battle']['teams']

                for team in teams:
                    players_tag = [member['tag'][1:] for member in team]
                    side = 1 if tag in players_tag else 2

                    for player in team:
                        player_info = client.get_profile(player['tag'][1:])
                        player_brawlers = sorted(player_info.brawlers, key= lambda x: x['trophies'], reverse=True)

                        for brawler in player_brawlers:
                            if brawler['id']==player['brawler']['id']:
                                gadget = 1 if ['gadgets'] else 0
                                star_power = 1 if brawler['star_powers'] else 0
                        
                        team_info = {
                            'player_tag': player['tag'],
                            'player_name': player['name'],
                            'player_trophies': player_info.trophies,
                            'player_top_brawler': player_brawlers[0]['name'],
                            'star_player': 1 if player['tag'][1:] == star_player_tag else 0,
                            'team': side,
                            'result': battle['battle']['result'],
                            'rank': None,
                            'brawler_id': player['brawler']['id'],
                            'brawler_name': player['brawler']['name'],
                            'brawler_power': player['brawler']['power'],
                            'brawler_star_power': star_power,
                            'brawler_gadget': gadget,
                            'brawler_trophies': player['brawler']['trophies'],
                        }

                        parsed_logs.append({**battle_info, **team_info})

        elif battle['event']['mode'] == 'duoShowdown':
            
            battle_info = {
                "battle_time": battle['battle_time'],
                "mode": battle['event']['mode'],
                "map": battle['event']['map'],
                "type": battle['battle']['type'],
                "duration": None,
            }

            teams = battle['battle']['teams']

            for team in teams:
                players_tag = [member['tag'][1:] for member in team]
                side = 1 if tag in players_tag else teams.index(team) + 2

                for player in team:
                    player_info = client.get_profile(player['tag'][1:])
                    player_brawlers = sorted(player_info.brawlers, key= lambda x: x['trophies'], reverse=True)

                    for brawler in player_brawlers:
                        if brawler['id']==player['brawler']['id']:
                            gadget = 1 if ['gadgets'] else 0
                            star_power = 1 if brawler['star_powers'] else 0
                    
                    team_info = {
                        'player_tag': player['tag'],
                        'player_name': player['name'],
                        'player_trophies': player_info.trophies,
                        'player_top_brawler': player_brawlers[0]['name'],
                        'star_player': 1 if player['tag'][1:] == star_player_tag else 0,
                        'team': side,
                        'result': None,
                        'rank': battle['battle']['rank'] if tag in players_tag else 0,
                        'brawler_id': player['brawler']['id'],
                        'brawler_name': player['brawler']['name'],
                        'brawler_power': player['brawler']['power'],
                        'brawler_star_power': star_power,
                        'brawler_gadget': gadget,
                        'brawler_trophies': player['brawler']['trophies'],
                    }

                    parsed_logs.append({**battle_info, **team_info})
    

    return parsed_logs

In [ ]:
# 듀오쇼다운 모드에서 rank 값이 잘 들어갔는지 확인합니다.
temp = pd.DataFrame(parse_single_user(rankers_tag_list[126]))
temp[~temp['rank'].isna()]

In [ ]:
temp.info()

In [ ]:
# 빈 데이터프레임을 만듭니다.
columns = pd.DataFrame(parse_single_user(rankers_tag_list[0])).columns
df = pd.DataFrame(columns=columns)
df

In [ ]:
# 1~200위 랭커의 정보를 가져와서 데이터프레임을 채워줍니다.
for ranker in rankers_tag_list:
    try:
        df = pd.concat([df, pd.DataFrame(parse_single_user(ranker))], axis=0, ignore_index=True)
    except:
        pass

df.head()

In [ ]:
df.info()

In [ ]:
df_origin = df.copy()

# 정규표현식을 사용하여 숫자, 영문만 남깁니다.
df["player_name"] = df["player_name"].str.replace(pat=r'[^1-9a-zA-Z]', repl=r'', regex=True)

df.head()

In [ ]:
df.to_csv(f'C:/Users/yenny/Desktop/{time}_rankers.csv', encoding='utf-8')